In [ ]:
import gurobipy as gp
from gurobipy import GRB, quicksum

def solve_fakenews_gurobi(arquivo_instancia):
    """
    Lê a instância do arquivo e resolve o problema de minimização da
    propagação de fake news com recursos de atraso, conforme a modelagem:
    
    - Minimiza o número total de servidores infectados em todos os cenários.
    - Cada servidor infectado deve ter o tempo de infecção t_{v,s} <= T,
      caso contrário (t_{v,s} > T) esse servidor não é considerado infectado.
    - Se um servidor está infectado, a infecção se propaga segundo:
         t_{v,s} >= t_{u,s} + t_{uv} + δ·x_u,  para cada arco (u,v)
    - Recursos (disponíveis em tempos β_i) podem ser alocados aos servidores,
      limitando a propagação.
    
    Retorna um dicionário com os resultados.
    """
    # 1. Ler os dados da instância
    delta, T, recursos, grafo = parse_instancia(arquivo_instancia)
    
    # Conjunto de servidores (vértices)
    V = sorted(grafo.keys())
    
    # Conjunto de arestas e dicionário de tempos de transmissão
    E = []
    t_uv = {}
    for u in V:
        for (v, t) in grafo[u]:
            E.append((u, v))
            t_uv[(u, v)] = t
    
    # Número de servidores e de recursos
    n = len(V)
    alpha = len(recursos)  # Recursos disponíveis
    I = range(alpha)       # Conjunto de índices dos recursos
    
    # Valor grande para big-M
    M = 1e7

    # 2. Criar o modelo
    m = gp.Model("MinFakeNewsPropagation")
    
    # 3. Variáveis de Decisão

    # 3.1. Estados de infecção: y[v,s] = 1 se o servidor v é infectado no cenário onde a fake news começa em s.
    y = m.addVars([(v, s) for v in V for s in V], vtype=GRB.BINARY, name="y")
    
    # 3.2. Propagação: z[u,v,s] = 1 se a fake news propaga de u para v no cenário s.
    z = m.addVars([(u, v, s) for (u, v) in E for s in V], vtype=GRB.BINARY, name="z")
    
    # 3.3. Tempo de infecção: t[v,s] = instante em que v é infectado no cenário s.
    t = m.addVars([(v, s) for v in V for s in V], vtype=GRB.CONTINUOUS, lb=0.0, name="t")
    
    # 3.4. Variáveis auxiliares para “cortar” infecções fora do prazo:
    # γ[v,s] = 1 indica que t[v,s] > T, forçando y[v,s] = 0.
    gamma = m.addVars([(v, s) for v in V for s in V], vtype=GRB.BINARY, name="gamma")
    
    # 3.5. Alocação de recursos:
    # x[v] = 1 se um recurso é alocado no servidor v.
    x = m.addVars(V, vtype=GRB.BINARY, name="x")
    
    # w[v,i] = 1 se o recurso i é alocado no servidor v.
    w = m.addVars([(v, i) for v in V for i in I], vtype=GRB.BINARY, name="w")
    
    # 4. Função Objetivo
    # Minimizar o total de servidores infectados em todos os cenários.
    m.setObjective(quicksum(y[v, s] for v in V for s in V), GRB.MINIMIZE)
    
    # 5. Restrições

    # 5.1. Para cada cenário, a fonte s está infectada no instante 0.
    for s in V:
        m.addConstr(y[s, s] == 1, name=f"infect_source_{s}")
        m.addConstr(t[s, s] == 0, name=f"time_source_{s}")
    
    # 5.2. Relação entre a propagação e a infecção:
    # Se a fake news se propaga de u para v, u deve estar infectado e v também será infectado.
    for (u, v) in E:
        for s in V:
            m.addConstr(z[u, v, s] <= y[u, s], name=f"propagate_from_{u}_{v}_s{s}")
            m.addConstr(y[v, s] >= z[u, v, s], name=f"infect_if_propagated_{u}_{v}_s{s}")
    
    # [A ativação dessa restrição ocasiona na inviabilidade do problema]
    # # 5.3. Propagação dos tempos de infecção:
    # # Se u está infectado, então para que v seja infectado deve ocorrer:
    # # t[v,s] >= t[u,s] + t_uv + δ·x[u]
    # for (u, v) in E:
    #     for s in V:
    #         m.addConstr(
    #             t[v, s] >= t[u, s] + t_uv[(u, v)] + delta * x[u],
    #             name=f"time_propagation_{u}_{v}_s{s}"
    #         )
    
    # 5.4. Limitação da infecção no tempo T utilizando a variável auxiliar γ:
    # Se t[v,s] ultrapassar T (γ[v,s] = 1), então y[v,s] deve ser 0.
    for v in V:
        for s in V:
            m.addConstr(
                y[v, s] <= M * (1 - gamma[v, s]),
                name=f"cut_infection_y_{v}_s{s}"
            )
            m.addConstr(
                t[v, s] <= T + M * gamma[v, s],
                name=f"cut_infection_t_{v}_s{s}"
            )
    
    # 5.5. Número máximo de recursos alocados:
    # O total de servidores com recurso não pode ultrapassar α.
    m.addConstr(quicksum(x[v] for v in V) <= alpha, name="max_resources")
    
    # 5.6. Cada servidor pode receber, no máximo, um recurso:
    for v in V:
        m.addConstr(x[v] == quicksum(w[v, i] for i in I), name=f"resource_allocation_{v}")
    
    # 5.7. Disponibilidade dos recursos:
    # Um recurso i só pode ser alocado se o servidor v for infectado após o tempo mínimo β_i.
    # Assim, para cada servidor v (e para cada cenário s, pois t[v,s] varia com o cenário),
    # temos: t[v,s] >= Σ_{i in I} β_i * w[v,i]
    for v in V:
        for s in V:
            m.addConstr(
                t[v, s] >= quicksum(recursos[i] * w[v, i] for i in I),
                name=f"resource_availability_{v}_s{s}"
            )
    
    # 6. Otimização do modelo
    m.optimize()
    
    # 7. Coleta da solução
    status = m.Status
    if status in [GRB.OPTIMAL, GRB.TIME_LIMIT, GRB.INTERRUPTED]:
        obj_val = m.ObjVal
        infectados_por_fonte = {}
        for s in V:
            infectados = []
            for v in V:
                if y[v, s].X > 0.5:
                    infectados.append(v)
            infectados_por_fonte[s] = infectados
        
        # Escolhe o cenário (fonte) que gera a menor contagem de infecções
        best_scenario = None
        min_infections = float('inf')
        for s in V:
            count = len(infectados_por_fonte[s])
            if count < min_infections:
                min_infections = count
                best_scenario = s
        
        resultado = {
            'status': status,
            'valor_objetivo': obj_val,
            'infectados_por_fonte': infectados_por_fonte,
            'best_scenario': best_scenario,
            'min_infections': min_infections
        }
        return resultado
    else:
        print(f"Modelo inviável (status = {status}).")
        return None

def parse_instancia(arquivo):
    """
    Formato da instância:
      - Primeira linha: <algum identificador> δ T
      - Segunda linha: m (número de grupos de recursos)
      - Próximas m linhas: β_i k  (tempo mínimo β_i e quantidade k desse recurso)
      - Demais linhas: "x1-y1 x2-y2 t_uv", definindo um arco de (x1,y1) para (x2,y2) com tempo t_uv.
    
    Retorna: δ, T, lista de recursos (cada recurso aparece com seu β_i) e o grafo (dicionário)
    """
    with open(arquivo, 'r') as f:
        linhas = f.readlines()
    
    # Primeira linha: ignoramos o primeiro valor e usamos δ e T
    _, delta, T = map(int, linhas[0].split())
    m_grupos = int(linhas[1])
    
    recursos = []
    for i in range(2, 2 + m_grupos):
        beta, k = map(int, linhas[i].split())
        recursos.extend([beta] * k)
    
    grafo = {}
    for linha in linhas[2 + m_grupos:]:
        if linha.strip():
            x1_y1, x2_y2, t_val = linha.strip().split()
            x1, y1 = map(int, x1_y1.split('-'))
            x2, y2 = map(int, x2_y2.split('-'))
            t_val = int(t_val)
            u = (x1, y1)
            v = (x2, y2)
            if u not in grafo:
                grafo[u] = []
            if v not in grafo:
                grafo[v] = []
            grafo[u].append((v, t_val))
    return delta, T, recursos, grafo

if __name__ == "__main__":
    # Altere o caminho da instância conforme necessário.
    instancia = "instances/fn1.dat"
    resultado = solve_fakenews_gurobi(instancia)
    
    if resultado is not None:
        print("Solução encontrada:")
        print(f" - Valor objetivo (total de infecções): {resultado['valor_objetivo']}")
        print(" - Infecções por cenário:")
        for s, infectados in resultado['infectados_por_fonte'].items():
            print(f"   * Fonte {s}: infectados = {infectados} (contagem = {len(infectados)})")
        print(f"Melhor cenário: Fonte {resultado['best_scenario']} com {resultado['min_infections']} infecções.")


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: Intel(R) Core(TM) i5-10210U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 829 rows, 768 columns and 3360 nonzeros
Model fingerprint: 0xfbb8e6ab
Variable types: 144 continuous, 624 integer (624 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+07]
Found heuristic solution: objective 12.0000000
Presolve removed 829 rows and 768 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 12 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.200000000000e+01, best bound 1.200000000000e+01, gap 0.0000%
Solução encontrada:
 - Valor